# Implicit User-Item Interaction based Recommender System
--------

## Introduction
--------
The growth of online sales platform rapidly increases and is accompanied by the increased number of products offered, there is a significant requirement to improve the customer experience. Imagine a new customer just signs up to a new platform with over hundreds, thousands or even hundreds of thousands of different products, how is his/ her experience going to be? Not so good, right?

Though there are a lot of products that he/ she might actually be interested in and would have purchased them, but if he/ she could not even find them, then we might have just lost a customer. Fortunately, advancement in Machine Learning studies has brought to life a solution to solve this problem, that is, a **recommender system**. The goal of this tool is to improve the customer experience through personalised item recommendations based on the past customer-item interaction (or what we refer to as implicit feedback).

## Objective
--------
After understanding the data by doing data exploratory analysis, we are ready to build the recommender system. Due to the nature of our sample data, which is considered an implicit feedback data, we can leverage a research documented on <a href="http://yifanhu.net/PUB/cf.pdf" target="_blank">this paper</a>, which is on _Collaborative Filtering on Implicit Feedback Datasets_.

## Approach
--------
Firstly, we need to understand the **challenge and characteristics** of building a recommender system with an implicit feedback data:
- we are unable to say or infer if a customer dislikes a product just because he/ she never purchases it
  The existence of past interaction between a user and a product can only tell us which items the user potentially likes, however it is not straight forward or even possible to figure out which items he/ she does not like. Unlike in explicit feedback data, where there is an information of user disliking a product, implicit feedback data does not have this. The reason of this non-existing interaction could be due to several reasons, for example, he/ she never came across that product, the product was unavailable or the price was too high (some customers wait till it's sale season)
- it is a noisy data
  Similar to the first point above, past user-item interaction does not necessarily mean he/ she likes that item. It could just be a gift purchased for someone else, a once-off purchase and he/ she ended up not liking it, etc.
- the number of interactions indicates confidence
  In explicit feedback data, the numeric value (ie. rating) indicates user's preferences with higher rating means higher preferences. On the other hand, the numeric value in implicit feedback data (ie. number of interactions/ purchases) describes frequency and higher frequency does not indicate higher preference. It only indicates confidence that he/ she likes that item, e.g. if someone keeps buying the same chocolate (recurring events), there is a higher confidence that he/ she likes it.
- faces cold start problem
  As we rely heavily on the past interactions between users and items, when there is a new user or item joining the online platform, it is almost impossible to give any recommendations. One way to address this is by the popularity based model, which is generic, meaning that the recommendations will be based on the top popular items and the same for all users.
  
There are 2 common techniques to build a recommender system: _content-based approach_ and _collaborative filtering_. As our dataset does not contain any user features or item features, it only makes sense to pick _collaborative filtering_ method (CF) that is reliant only on the historical user-item interactions, without the need of user features or item features. Essentially, **CF** analyses the relationships between users and interdependencies among products and finally, identifies user-item associations (or known as _latent factors_).

## Implicit Collaborative Filtering - Method
--------
Based on <a href="http://yifanhu.net/PUB/cf.pdf" target="_blank">this paper</a> written by **Hu, Koren, and Volinsky**, this is the summary of the method we are going to apply to our data to build the CF based recommender system.

Basically,  from our dataset we can construct a large user-item interaction $m . n$ matrix, where $m$ is the number of unique users and $n$ the number of unique items, accounting for all the possible user-item pairs. This means, our matrix will have a lot of 0 values denoting the non-existing user-item interactions in the data. What we are trying to do is to decompose this huge matrix into user-factors $x_{u}$ and item-factors $y_{i}$ vectors, which their dot product will predict the user $u$ preference to a specific item $i$.

Each of the user-item interaction will be represented by $r_{ui}$, ie. the number of times user $u$ interacted with item $i$. This measures the confidence indicator of user liking a particular item and hence, the preference $p_{ui}$ variable is introduced and defined as:  
$
    p_{ui}= 
\begin{cases}
    1,& r_{ui} > 0\\
    0,& r_{ui} = 0
\end{cases}
$  

It is worth noting that this preference variable is associated with **highly varying confidence levels**:
- considering it is an implicit feedback data, $r_{ui} = 0$ denotes low confidence level and it is due to the fact that non-existing user-item interaction may be caused by many reasons (e.g. unawareness of the item, unavailability of the item, etc)
- interacting with an item does not always mean that the user likes the item as it could be for a gift (and he does not actually like it himself)

In general, as the number of user-item interaction increases, the indication that the user likes the item gets higher - higher confidence. Because of this concept, the confidence $c_{ui}$ variable is introduced and defined as:  
$c_{ui} = 1 + \alpha r_{ui}$

where $c_{ui}$ measures the confidence in observing $p_{ui}$, ie. the confidence that we have on user's preference to a particular item and $\alpha$ as the rate of increase for $c_{ui}$ for every increase in $r_{ui}$. This way, we will always have a minimum confidence in the preference $p_{ui}$ and the confidence increases as the number of interaction $r_{ui}$ increases.  

As mentioned above, the goal is to compute user-factors and item-factors vectors to define user-item association and this means our predicted user preference on a specific item can defined as:  
$\hat{p}_{ui} = x_u^Ty_i$

where $x_u$ represents user-factors, $y_i$ item-factors and $\hat{p}_{ui}$ the assumed preference measure of user $u$ to item $i$. What the 2 vectors $x_u$ and $y_i$ are trying to achieve is to define the common latent factors so they can be directly compared.

Finally, the objective is to compute the **user-factors** and **item-factors** vectors by minimising the following **cost function**:  
$cost = \sum_{u, i} {c_{ui}(p_{ui} - x_u^Ty_i)^2} + \lambda (\sum_u \|x_u\|^2 + \sum_i \|y_i\|^2)$  
where $\lambda$ is the **regularisation term**, which is crucial to prevent _overfitting_ to our training data.

By differentiation, the expressions to calculate both $x_{u}$ and $y_{i}$ to minimise the above **cost function** are as follows:  
$x_{u} = (Y^TY + Y^T(C^u - I)Y + \lambda I)^{-1}Y^TC^up(u)$  
$y_{i} = (X^TX + X^T(C^i - I)X + \lambda I)^{-1}X^TC^ip(i)$

We are going to use the Alternative Least Square (ALS) method to solve the equations. The ALS is taken from the `implicit` package available from <a href="https://github.com/benfred/implicit" target="_blank">here</a>.

After the $x_{u}$ and $y_{i}$ are calculated, we can recommend top $K$ items to a particular user based on $K$ highest values of $\hat{p}_{ui}$ as defined above as the predicted user preference to a certain item.

## Data Treatment
--------
The dataset used in the experiment conducted by **Hu, Koren, and Volinsky** had the following information:
- user_id (anonymised, same group of users for the entire experiment)
- program_id (over 17,000 unique programs)
- timestamp (over 4 weeks period)
- non-zero records, $r_{ui}$ were over 32 million values

The above dataset was used as their _training_ set, whereas the _test_ set was an identical dataset which covers the following 1 week period after the 4 weeks period in the _training_ dataset. Comparing this to the dataset we have, we cannot follow their approach in splitting _training_ and _test_ sets. Fundamentally, the reason is because we have new users and items across the time period covered in our dataset.

Considering this known property of our dataset, the following options could be used for splitting the data into _training_ and _test_ sets:
- for the _training_ set, for each user $u$, retain only $n$ unique items the user had interactions with in earlier times, "masking" or excluding the remaining items in the later times and for the _test_ set, it will be the full dataset so we can compare if our recommended items were actually purchased or not
- for the _training_ set, subset the dataset up to a certain time period and build the recommender system based on this and the remaining data will be the _test_ set; in the case where it is a new user who is not part of the _training_ set, we provide top 10 items using **popularity-based** method
- for the _training_ set, randomly choose a % of the existing user-item interactions to be replaced by **0** (as if user never interacted with that item) to build the model and for the _test_ set, same as the first option above, it will be the full dataset

Our data exploratory analysis provides the following facts about our data:
- new users appearing across the time period
- user tenure analysis demonstrates that over 75% of our users had a tenure of less than 1 year
  This means that seasonal items purchased by those users will not be purchased again in a short period of time.
  
Based on these conditions, option 3 is chosen as the option to split our dataset into _training_ and _test_ sets.

## Performance Measure
--------
There are 3 x performance measures selected to evaluate our recommender system:
- **recall**  
  Recall is the ratio between the number of items that were correctly predicted by the recommender system and the actual items purchased by users (how many recommendations were relevant to the user). For instance, out of the items our system recommends, 4 of them were actually purchased by the user out of the total 8 items purchased by the user; in this case, our recall measure is 4 / 8 = 50%.
- **precision**  
  Precision is the ratio between the number of items that were correctly predicted by the recommender system and the number of items recommended to the user (how precise is our recommender system). For instance, out of the 10 items our system recommends, 4 of them were actually purchased by the user out of the total 8 items purchased by the user; in this case, our precision is 4 / 10 = 40%
  It is worth noting that we might have a low precision score due to the fact that some users might not have bought 10 items yet (considering we always give 10 items, regardless).
- **masked predicted ratio**  
  This measures if the user ended up buying any of the items our system recommended, ie. focusing on the users in the training set where some of their purchased items were masked to build the model. In other words, we want to calculate how many masked items of a few users were recommended by our system. This could arguably be the most important measure out of the three performance measures mentioned here, because the ability to recommend items which user ends up purchasing is very beneficial.
  
## Performance Results
--------
Below are the results of our performance measure testing:
- recall measure: **59.1%**
- precision measure: **34.7%**
- area under the ROC curve: **39.9%**

## Model Optimisation
--------
The initial model used the parameter values as per the referenced paper above:
- alpha $\alpha = 40$
- factors = 100
- regularization term $\lambda = 0.01$
- iterations = 10

### Parameters testing
##### Alpha
We optimised this value by running tests to build the recommender system using a range of alpha values and measure their corresponding `recall_measure`. It was found that an alpha value of 590 was good for our model.

##### Factors
Using the default value of alpha of 40 and new value of alpha of 590, we tested for parameter factors and found that increasing the factors to 200 yielded in a slight improvement on the recall_measure.

##### Regularization term
Same method as testing for `factors`, the tests were run using default values and adjusted values of alpha and factors. It was found that our default value of 0.01 already gives a good result.

##### Iterations
Same method as above, the tests were run using default values and adjusted values of alpha, factors and regularization term. It was found that our default value of 10 already gives a good result.

### Chosen parameters values
Though, slight improvement was observed on precision and recall metrics (around 2%), changing the parameter values from the default values decreased the missing predicted ratio metric considerably (from around 40% down to 20%). As mentioned earlier that missing predicted ratio metric is arguably the most important measure, it was decided to keep the parameter values as the initial ones - which are the same as the ones used by **Hu, Koren, and Volinsky** in their experiment.

## Future Improvements
--------
The fact that a user does not interact with an item does not mean he/ she doesn't like it. It might mean that the user was not aware of the item or the item was not available at the time. Additionally, it could also be due to seasonality - which our exploratory analysis seems to suggest time element has an impact on user's behaviour. This leads to potential enhancement of the model, which is to address the tendency of a user to buy items on certain times. Moreover, as a matter of fact, certain items are more popular in different times of day/ seasons of the year (e.g. ice cream at night, breakfast bar in the morning, winter jackets in cold months, etc.)